In [1]:
#Invincible_Predictors_Exploratory_Data_Analysis

#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#Customizing Output options
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 50)
sns.set(color_codes=True)

In [3]:
#Importing CSV Files
air_reserve = pd.read_csv('air_reserve.csv',parse_dates=['visit_datetime', 'reserve_datetime'])
hpg_reserve = pd.read_csv('hpg_reserve.csv',parse_dates=['visit_datetime', 'reserve_datetime'])
air_store_info = pd.read_csv('air_store_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
date_info = pd.read_csv('date_info.csv',parse_dates=['calendar_date'])
train = pd.read_csv('train.csv',parse_dates=['visit_date'])
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
#dropping the day of week column as we would already extract it from datetime value.
date_info.drop(columns=['day_of_week'],inplace=True)

In [5]:
#renaming date column so that it will be useful while merging the holiday flag with training data.
date_info.rename(columns={'calendar_date':'visit_date'},inplace=True)
date_info

,visit_date,holiday_flg
0,2016-01-01,1
1,2016-01-02,1
2,2016-01-03,1
3,2016-01-04,0
4,2016-01-05,0
...,...,...
512,2017-05-27,0
513,2017-05-28,0
514,2017-05-29,0
515,2017-05-30,0


In [6]:
#Adding air_store_id column and performing inner join to discard those rows which does not have air_store_id associated with it.
hpg_reserve_storeinfo = hpg_reserve.merge(store_id_relation, how='inner',on='hpg_store_id')
hpg_reserve_storeinfo

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926
...,...,...,...,...,...
28178,hpg_0f78ef8d1e4b23d8,2017-04-01 21:00:00,2017-03-10 20:00:00,2,air_84f6876ff7e83ae7
28179,hpg_0f78ef8d1e4b23d8,2017-04-02 17:00:00,2017-04-02 14:00:00,2,air_84f6876ff7e83ae7
28180,hpg_0f78ef8d1e4b23d8,2017-04-08 19:00:00,2017-04-03 12:00:00,3,air_84f6876ff7e83ae7
28181,hpg_0f78ef8d1e4b23d8,2017-04-08 20:00:00,2017-03-09 07:00:00,4,air_84f6876ff7e83ae7


In [7]:
#Adjusting the columns and making it align with air_reserve table so that it can be appended later.
hpg_reserve_storeinfo = hpg_reserve_storeinfo[['air_store_id','visit_datetime','reserve_datetime','reserve_visitors']]
hpg_reserve_storeinfo

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_db80363d35f10926,2016-01-01 19:00:00,2016-01-01 15:00:00,4
1,air_db80363d35f10926,2016-01-02 19:00:00,2016-01-02 14:00:00,2
2,air_db80363d35f10926,2016-01-03 18:00:00,2016-01-02 20:00:00,6
3,air_db80363d35f10926,2016-01-06 20:00:00,2016-01-04 22:00:00,3
4,air_db80363d35f10926,2016-01-11 18:00:00,2016-01-11 14:00:00,2
...,...,...,...,...
28178,air_84f6876ff7e83ae7,2017-04-01 21:00:00,2017-03-10 20:00:00,2
28179,air_84f6876ff7e83ae7,2017-04-02 17:00:00,2017-04-02 14:00:00,2
28180,air_84f6876ff7e83ae7,2017-04-08 19:00:00,2017-04-03 12:00:00,3
28181,air_84f6876ff7e83ae7,2017-04-08 20:00:00,2017-03-09 07:00:00,4


In [8]:
#Merging Air reservation data and HPG reservation data for all air_store_id that are available in both tables.
#This is basically appending the hpg data after air data. Because each reservation is done separately, we'll combine
#it date wise later.
reservations = air_reserve.append(hpg_reserve_storeinfo)
reservations

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5
...,...,...,...,...
28178,air_84f6876ff7e83ae7,2017-04-01 21:00:00,2017-03-10 20:00:00,2
28179,air_84f6876ff7e83ae7,2017-04-02 17:00:00,2017-04-02 14:00:00,2
28180,air_84f6876ff7e83ae7,2017-04-08 19:00:00,2017-04-03 12:00:00,3
28181,air_84f6876ff7e83ae7,2017-04-08 20:00:00,2017-03-09 07:00:00,4


In [9]:
#Time of booking and time of reservation.
reservations["visit_date"] = pd.DatetimeIndex(reservations['visit_datetime']).date
reservations["visit_time"] = pd.DatetimeIndex(reservations['visit_datetime']).hour
reservations["reserve_time"] = pd.DatetimeIndex(reservations['reserve_datetime']).hour 

#Time difference between planned visit time and booking time.
reservations["Time_Difference"] = (reservations['visit_datetime'] - reservations['reserve_datetime']).dt.seconds / 3600
reservations

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,visit_time,reserve_time,Time_Difference
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,19,16,3.0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,19,19,0.0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,19,19,0.0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,20,16,4.0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,20,1,19.0
...,...,...,...,...,...,...,...,...
28178,air_84f6876ff7e83ae7,2017-04-01 21:00:00,2017-03-10 20:00:00,2,2017-04-01,21,20,1.0
28179,air_84f6876ff7e83ae7,2017-04-02 17:00:00,2017-04-02 14:00:00,2,2017-04-02,17,14,3.0
28180,air_84f6876ff7e83ae7,2017-04-08 19:00:00,2017-04-03 12:00:00,3,2017-04-08,19,12,7.0
28181,air_84f6876ff7e83ae7,2017-04-08 20:00:00,2017-03-09 07:00:00,4,2017-04-08,20,7,13.0


In [10]:
#Grouping the reservations by air_store_id and date columns because training dataset contains visitors for a 
#restaurant for respective dates.
reservations_by_date = reservations[['air_store_id','visit_date','reserve_visitors']].groupby(['air_store_id','visit_date'],as_index=False).sum()

#Taking the mean of time difference, visit time and reserve time grouped by store id and date.
reservations_by_date['Time_Difference'] = reservations[['air_store_id','visit_date','Time_Difference',]].groupby(['air_store_id','visit_date'],as_index=False).mean()['Time_Difference']
reservations_by_date['visit_time']= reservations[['air_store_id','visit_date','visit_time',]].groupby(['air_store_id','visit_date'],as_index=False).mean()['visit_time']
reservations_by_date['reserve_time']= reservations[['air_store_id','visit_date','reserve_time',]].groupby(['air_store_id','visit_date'],as_index=False).mean()['reserve_time']
reservations_by_date.visit_date = pd.to_datetime(reservations_by_date.visit_date)

reservations_by_date

,air_store_id,visit_date,reserve_visitors,Time_Difference,visit_time,reserve_time
0,air_00a91d42b08b08d9,2016-01-14,2,5.0,19.0,14.0
1,air_00a91d42b08b08d9,2016-01-15,4,22.0,18.0,20.0
2,air_00a91d42b08b08d9,2016-01-16,2,22.0,18.0,20.0
3,air_00a91d42b08b08d9,2016-01-22,2,10.0,18.0,8.0
4,air_00a91d42b08b08d9,2016-01-29,5,0.0,18.0,18.0
...,...,...,...,...,...,...
42188,air_fea5dc9594450608,2017-04-22,4,3.0,22.0,19.0
42189,air_fea5dc9594450608,2017-04-25,4,20.0,18.0,22.0
42190,air_fea5dc9594450608,2017-04-28,6,0.0,19.0,19.0
42191,air_fea5dc9594450608,2017-05-20,12,21.0,19.0,22.0


In [27]:
#adding store info to the training dataset from air_store_info table as it contains data for all the air stores.
train_temp = train.merge(air_store_info, how='left', on='air_store_id')

In [34]:
#preparing final training dataset by merging reservation data to the train data.
train_data = train_temp.merge(reservations_by_date,how='left',on=['air_store_id','visit_date'])

#As we would have many null values, because most of the restaurants in training data set does not have reservation data.
train_data.fillna(value=0,inplace=True)

#We do not require latitude longitude information as we extracted the city ward and neighborhood.
train_data.drop(columns=['latitude','longitude'],inplace=True)

#Extracting year month weekday as new features
train_data["visit_year"] = pd.DatetimeIndex(train_data['visit_date']).year
train_data["visit_month"] = pd.DatetimeIndex(train_data['visit_date']).month
train_data["visit_weekday"] = pd.DatetimeIndex(train_data['visit_date']).weekday

#Extracting city ward neighbourhood as new features
train_data['city'] = train_data['air_area_name'].str.split().str[0]
train_data['ward'] = train_data['air_area_name'].str.split().str[1]
train_data['neighborhood'] = train_data['air_area_name'].str.split().str[2]

#Add holiday flag from date info table
train_data = train_data.merge(date_info,how='left',on='visit_date')

#Making all object type columns as categorical columns so that it may help in plotting.
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        train_data[col] = train_data[col].astype('category')

train_data

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,reserve_visitors,Time_Difference,visit_time,reserve_time,visit_year,visit_month,visit_weekday,city,ward,neighborhood,holiday_flg
0,air_e3020992d5fe5dfd,2016-07-01,21,Italian/French,Tōkyō-to Chūō-ku Tsukiji,0.0,0.0,0.0,0.0,2016,7,4,Tōkyō-to,Chūō-ku,Tsukiji,0
1,air_e3020992d5fe5dfd,2016-07-02,19,Italian/French,Tōkyō-to Chūō-ku Tsukiji,0.0,0.0,0.0,0.0,2016,7,5,Tōkyō-to,Chūō-ku,Tsukiji,0
2,air_e3020992d5fe5dfd,2016-07-04,8,Italian/French,Tōkyō-to Chūō-ku Tsukiji,0.0,0.0,0.0,0.0,2016,7,0,Tōkyō-to,Chūō-ku,Tsukiji,0
3,air_e3020992d5fe5dfd,2016-07-05,11,Italian/French,Tōkyō-to Chūō-ku Tsukiji,0.0,0.0,0.0,0.0,2016,7,1,Tōkyō-to,Chūō-ku,Tsukiji,0
4,air_e3020992d5fe5dfd,2016-07-06,16,Italian/French,Tōkyō-to Chūō-ku Tsukiji,0.0,0.0,0.0,0.0,2016,7,2,Tōkyō-to,Chūō-ku,Tsukiji,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239668,air_4ce7b17062a1bf73,2017-03-30,15,Italian/French,Tōkyō-to Minato-ku Shibakōen,0.0,0.0,0.0,0.0,2017,3,3,Tōkyō-to,Minato-ku,Shibakōen,0
239669,air_4ce7b17062a1bf73,2017-03-31,14,Italian/French,Tōkyō-to Minato-ku Shibakōen,0.0,0.0,0.0,0.0,2017,3,4,Tōkyō-to,Minato-ku,Shibakōen,0
239670,air_4ce7b17062a1bf73,2017-04-01,9,Italian/French,Tōkyō-to Minato-ku Shibakōen,0.0,0.0,0.0,0.0,2017,4,5,Tōkyō-to,Minato-ku,Shibakōen,0
239671,air_4ce7b17062a1bf73,2017-04-03,11,Italian/French,Tōkyō-to Minato-ku Shibakōen,0.0,0.0,0.0,0.0,2017,4,0,Tōkyō-to,Minato-ku,Shibakōen,0


In [33]:
#Just cross checking if all the restaurants in submission csv in present in our training dataset or not.
pd.DataFrame(sample_submission.id.str.rsplit('_',1))['id'].str[0].isin(train_data.air_store_id).all()

True

In [35]:
train_data.columns

Index(['air_store_id', 'visit_date', 'visitors', 'air_genre_name',
       'air_area_name', 'reserve_visitors', 'Time_Difference', 'visit_time',
       'reserve_time', 'visit_year', 'visit_month', 'visit_weekday', 'city',
       'ward', 'neighborhood', 'holiday_flg'],
      dtype='object')

In [36]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239673 entries, 0 to 239672
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   air_store_id      239673 non-null  category      
 1   visit_date        239673 non-null  datetime64[ns]
 2   visitors          239673 non-null  int64         
 3   air_genre_name    239673 non-null  category      
 4   air_area_name     239673 non-null  category      
 5   reserve_visitors  239673 non-null  float64       
 6   Time_Difference   239673 non-null  float64       
 7   visit_time        239673 non-null  float64       
 8   reserve_time      239673 non-null  float64       
 9   visit_year        239673 non-null  int64         
 10  visit_month       239673 non-null  int64         
 11  visit_weekday     239673 non-null  int64         
 12  city              239673 non-null  category      
 13  ward              239673 non-null  category      
 14  neig

In [37]:
train_data.to_pickle("./train_data_updated.pkl")


In [ ]:
#Below code is for later part when we will test the model for prediction.

In [ ]:
#test_data.to_pickle("./test_data.pkl")

In [ ]:
#sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
#sample_submission['air_store_id'] = sample_submission['id'].str.rsplit('_',1).str[0]
#sample_submission['visit_date'] = sample_submission['id'].str.rsplit('_',1).str[1]
#sample_submission.visit_date = pd.to_datetime(sample_submission.visit_date)
#sample_submission["visit_year"] = pd.DatetimeIndex(sample_submission['visit_date']).year
#sample_submission["visit_month"] = pd.DatetimeIndex(sample_submission['visit_date']).month
#sample_submission["visit_weekday"] = pd.DatetimeIndex(sample_submission['visit_date']).weekday
#sample_submission = sample_submission.merge(air_store_info, how='left', on='air_store_id')
#sample_submission.drop(columns=['latitude','longitude'],inplace=True)
#sample_submission['city'] = sample_submission['air_area_name'].str.split().str[0]
#sample_submission['ward'] = sample_submission['air_area_name'].str.split().str[1]
#sample_submission['neighborhood'] = sample_submission['air_area_name'].str.split().str[2]

In [ ]:
#test_data = sample_submission[['air_store_id', 'visit_date', 'visitors', 'air_genre_name',
#       'air_area_name', 'visit_year', 'visit_month', 'visit_weekday', 'city',
#       'ward', 'neighborhood']]

In [ ]:
#test_data = test_data.merge(date_info,how='left',on='visit_date')

In [ ]:
#for col in test_data.columns:
#    if test_data[col].dtype == 'object':
 #       test_data[col] = test_data[col].astype('category')